In [5]:
#Make Kenpom the standard team names
import pandas as pd
import numpy as np

KP = pd.read_csv('step01_kenpom0824.csv')
repair = pd.read_csv('step04_repair.csv')
LF = list(repair['tofix'])
LR = list(repair['replacewith'])
KP['Team'] = KP['Team'].replace(LF,LR)
KP = KP[KP['Team']!='out']
sn = KP[KP['Year']>2020].sort_values('Team')['Team'].unique()
KPN = KP.sort_values('Team')['Team'].unique()
KP2fix = list(set(KPN) - set(sn))
KP.to_csv('step01_kenpom0824.csv',index=False)
pd.DataFrame(sn).to_csv('asn.csv',index=False)
KP2fix

[]

In [6]:
# Make ESPN BPI Name match Kep Pom
BPI = pd.read_csv('step02_espnbpi0824.csv')
repair = pd.read_csv('step04_repair.csv')
LF = list(repair['tofix'])
LR = list(repair['replacewith'])
BPI['Team'] = BPI['Team'].replace({'State':'St.'}, regex=True)
BPI['Team'] = BPI['Team'].replace(LF,LR)
BPI = BPI[BPI['Team']!='out']
BPIN = BPI['Team'].unique()
BPI2fix = list(set(BPIN) - set(sn))

BPI.to_csv('step02_espnbpi0824.csv',index=False)
BPI2fix

[' Le Moyne']

In [3]:
# Make Baskeball Reference Team Names match Ken Pom
BR = pd.read_csv("step03_br0824.csv")
repair = pd.read_csv('step04_repair.csv')
LF = list(repair['tofix'])
LR = list(repair['replacewith'])
BR['Team'] = BR['Team'].replace({'State':'St.'}, regex=True)
BR['Team'] = BR['Team'].replace(LF,LR)
BR = BR[BR['Team']!='out']
BRN = BR['Team'].unique()
BR2fix = list(set(BRN) - set(sn))

BR.to_csv('step03_br0824.csv',index=False)
BR2fix

[]

In [ ]:
# Make Bart Reference Team Names match Ken Pom
bartdata = pd.read_csv("step03a_bart0824.csv")
repair = pd.read_csv('step04_repair.csv')
LF = list(repair['tofix'])
LR = list(repair['replacewith'])
bartdata['Team'] = bartdata['Team'].replace({'State':'St.'}, regex=True)
bartdata['Team'] = bartdata['Team'].replace(LF,LR)
bartdata = bartdata[bartdata['Team']!='out']
bartdataN = bartdata['Team'].unique()
bart2fix = list(set(bartdataN) - set(sn))

bartdata.to_csv('step03_br0824.csv',index=False)
bart2fix

[]

In [4]:
import pandas as pd
# Make the history AGorite team names match Ken Pom
AG = pd.read_csv("step04_FUHistory.csv")
AG['AFTeam'] = AG['AFTeam'].replace(LF,LR)
AG = AG[AG['AFTeam']!='out']
AGN = AG['AFTeam'].unique()
AG2fix = list(set(AGN) - set(sn))
AG2fix

[]

In [5]:
# Make the history Underdog team names match Ken Pom
AG['AUTeam'] = AG['AUTeam'].replace(LF,LR)
AG = AG[AG['AUTeam']!='out']
AGN = AG['AUTeam'].unique()
AG2fix = list(set(AGN) - set(sn))

AG.to_csv('step04_FUHistory.csv',index=False)
AG2fix


[]

In [6]:
# Compute Seed History
AG = pd.read_csv('step04_FUHistory.csv')
#AG = AG[AG['Year']<2023]
LG = AG[AG['Round']==6]
CS = pd.DataFrame({'Round','Seed'})

for row in range(len(LG)):
    if LG.iloc[row]["AFScore"]>LG.iloc[row]["AUScore"]:
        ws = LG.iloc[row]["AFSeed"]

    else:
        ws = LG.iloc[row]["AUSeed"]
    CS.loc[row,'Round'] = 'W'
    CS.loc[row,'Seed'] = ws


cs = pd.crosstab(index=CS['Seed'],columns=CS['Round'])
fs = pd.crosstab(index=AG['AFSeed'],columns=AG['Round'])
us = pd.crosstab(index=AG['AUSeed'],columns=AG['Round'])
ufs = fs.add(us, fill_value=0).astype(int).drop([0],axis=1)
df = ufs.add(cs, fill_value=0).fillna(0).astype(int)
nos = df.max().max()
df['Exp Wins'] = (df.sum(axis=1)-nos)/nos
df.to_csv('step04_SeedHistory.csv',index=False) 
#Build PASE

# table of wins
fw = pd.crosstab(index=AG['AFTeam'],columns=AG['Year'])
uw = pd.crosstab(index=AG['AUTeam'],columns=AG['Year'])

# How many wins did the team acutally get per year if they were in the tournament
cw = fw.add(uw, fill_value=0).fillna(0).astype(int) - 1
cw = cw.replace(-1, '')

# table of seeds
AG1 = AG[AG['Round']==1]
fs = AG1.pivot(index='AFTeam', columns='Year', values = 'AFSeed').fillna(0).astype(int)
us = AG1.pivot(index='AUTeam', columns='Year', values = 'AUSeed').fillna(0).astype(int)

cs = fs.add(us, fill_value=0).fillna(0).astype(int)
cs = cs.replace(0, '')

# table of expected wins
sh =pd.read_csv('step04_SeedHistory.csv')
ew = pd.DataFrame({'Seed':np.arange(1,17),'Exp Wins':sh['Exp Wins']})

# ewty is expected number of wins by team and by year
ewty = cs.replace(list(ew['Seed']),list(ew['Exp Wins']))
# fill in zeros into the blank spaces
ewz = (ewty * -1).replace(r'^\s*$', np.nan, regex=True).fillna(0)

cwz = cw.replace(r'^\s*$', np.nan, regex=True).fillna(0)
pase = cwz.add(ewz).replace(0, np.nan)
pase['PASE']=pase.mean(axis=1)
pase = pase.iloc[:,-1:]

pase.to_csv('step05_PASE.csv')   


C:\Users\harmk\AppData\Local\Temp\ipykernel_19440\2000385893.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ewz = (ewty * -1).replace(r'^\s*$', np.nan, regex=True).fillna(0)
C:\Users\harmk\AppData\Local\Temp\ipykernel_19440\2000385893.py:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cwz = cw.replace(r'^\s*$', np.nan, regex=True).fillna(0)


In [7]:
# Make sure PASE team names are matched to Ken Pom
pase = pd.read_csv("step05_PASE.csv")
pase.columns = ['Team','PASE']
pase['Team'] = pase['Team'].replace(LF,LR)
pase = pase[pase['Team']!='out']
paseN = pase['Team'].unique()
pase2fix = list(set(paseN) - set(sn))

pase.to_csv('step05_PASE.csv',index=False)
pase2fix

[]

In [8]:
#Combine all the datframes
KPBPI = KP.merge(BPI,how='left', on=['Year','Team'])
KPBPIBR = KPBPI.merge(BR, how = 'left', on=['Year','Team'])
KPBPIBRP = KPBPIBR.merge(pase, how = 'left', on = ['Team'])
KPBPIBRP.to_csv('step06_AllStats.csv',index=False)

In [9]:
#Attach stats to Tournament Games
AG = pd.read_csv('step04_FUHistory.csv')
AG = AG[AG['Year']>=2008]
AGstats = AG.merge(KPBPIBRP, left_on = ['Year','AFTeam'], right_on = ['Year','Team'], how = 'left')
AGstatsandU = AGstats.merge(KPBPIBRP, left_on = ['Year','AUTeam'], right_on = ['Year','Team'], how = 'left')
AGstatsandU = AGstatsandU.drop(['Team_x','Team_y'],axis=1)
AGstatsandU.to_csv('step07_FUStats.csv',index=False)